In [1]:
# 1. 필요한 라이브러리
from typing import TypedDict
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables import RunnableLambda
import os
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))

C:\Users\Admin\AppData\Local\Temp\ipykernel_9668\1600758116.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))


In [2]:
# 2. 상태 정의
class QAState(TypedDict):
    question: str
    route_label: str  # ← 이름 변경
    retrieved: str
    answer: str

In [8]:
# 3. LLM으로 분기 판단 (route_label만 업데이트)
def llm_router_update(state: QAState) -> QAState:
    prompt = f"""
    사용자의 질문을 보고 아래 중 어떤 유형인지 판단하세요:

    - general: 일반 지식이나 개념
    - web: 최근 뉴스, 최신 정보
    - db: 회의록, 내부 데이터 요약

    질문: "{state['question']}"

    위 3가지 중 하나의 단어(general/web/db)만 반환하세요.
    """
    route = llm.invoke(prompt).content.strip().lower()
    print(f"🔀 [LLM 라우팅 판단] → {route}")
    state["route_label"] = route
    return state

# 4. route_label 값을 조건 분기에 사용
def return_route_key(state: QAState) -> str:
    return state["route_label"]


In [12]:
# 5. 각 실행 노드 정의
def general_llm(state: QAState) -> QAState:
    response = llm.invoke(f"질문: {state['question']}\n\n위 질문에 답해주세요.")
    state["answer"] = response.content.strip()
    return state

def web_search(state: QAState) -> QAState:
    state["retrieved"] = "[웹 검색 결과] 오늘의 주요 뉴스를 요약했습니다."
    return state

def db_search(state: QAState) -> QAState:
    state["retrieved"] = "[DB 검색 결과] 회의 내용을 요약했습니다."
    return state

def generate_answer(state: QAState) -> QAState:
    if state.get("retrieved"):
        state["answer"] = f"[검색 기반 응답] {state['retrieved']}"
    return state


In [ ]:
# 6. LangGraph 구성 (judge_route → 조건 분기)
def build_news_router_graph():
    builder = StateGraph(QAState)

    # 노드 등록
    builder.add_node("judge_route", RunnableLambda(llm_router_update))
    builder.add_node("general", RunnableLambda(general_llm))
    builder.add_node("web", RunnableLambda(web_search))
    builder.add_node("db", RunnableLambda(db_search))
    builder.add_node("generate", RunnableLambda(generate_answer))

    # 시작점
    builder.set_entry_point("judge_route")

    # 조건 분기: judge_route → general/web/db
    builder.add_conditional_edges(
        "judge_route",        # 바로 여기서 분기
        return_route_key,
        {
            "general": "general",
            "web": "web",
            "db": "db"
        }
    )

    # 후처리 흐름
    builder.add_edge("general", "generate")
    builder.add_edge("web", "generate")
    builder.add_edge("db", "generate")

    builder.set_finish_point("generate")

    return builder.compile()


In [14]:
# 7. 테스트 실행
graph = build_news_router_graph()

questions = [
    "에이전트란 무엇인가요?",
    "오늘의 주요 뉴스 알려줘",
    "지난 회의 요약해줘"
]

for q in questions:
    print(f"\n❓ 질문: {q}")
    result = graph.invoke({
        "question": q,
        "route_label": "",
        "retrieved": "",
        "answer": ""
    })
    print(f"✅ 응답: {result['answer']}")


❓ 질문: 에이전트란 무엇인가요?
🔀 [LLM 라우팅 판단] → general
✅ 응답: "에이전트"라는 용어는 다양한 맥락에서 사용될 수 있지만, 일반적으로 다음과 같은 의미를 가집니다.

1. **일반적인 의미**: 에이전트는 어떤 일을 대신 수행하는 대리인이나 대행자를 의미합니다. 예를 들어, 부동산 에이전트는 고객을 대신하여 부동산 거래를 중개하는 역할을 합니다.

2. **컴퓨터 과학 및 인공지능**: 에이전트는 특정 환경에서 자율적으로 행동하고 결정을 내릴 수 있는 프로그램이나 시스템을 의미합니다. 예를 들어, 소프트웨어 에이전트는 사용자의 요구를 이해하고 그에 맞는 작업을 수행하는 프로그램입니다.

3. **비즈니스 및 마케팅**: 에이전트는 특정 제품이나 서비스를 홍보하고 판매하는 역할을 하는 사람이나 조직을 지칭합니다. 예를 들어, 스포츠 에이전트는 운동선수의 계약을 관리하고 협상하는 역할을 합니다.

4. **법률**: 법률에서 에이전트는 다른 사람(주체)을 대신하여 법적 행위를 수행할 수 있는 권한을 가진 사람을 의미합니다.

이처럼 "에이전트"는 그 사용되는 분야에 따라 다양한 의미를 가질 수 있습니다.

❓ 질문: 오늘의 주요 뉴스 알려줘
🔀 [LLM 라우팅 판단] → web
✅ 응답: [검색 기반 응답] [웹 검색 결과] 오늘의 주요 뉴스를 요약했습니다.

❓ 질문: 지난 회의 요약해줘
🔀 [LLM 라우팅 판단] → db
✅ 응답: [검색 기반 응답] [DB 검색 결과] 회의 내용을 요약했습니다.
